# CatBoost  
In this notebook, I implement a CatBoost model based on outfit tags.  

This approach failed because it returned a score of 0 for all evaluation metrics.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/MyDrive/RecSys_206894495'
%run /content/drive/MyDrive/RecSys_206894495/models/evaluate_models.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
empty json


In [ ]:
!pip install pyarrow
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
file_path_outfit = path + "/models/val_outfit.json"
file_path_group = path + "/models/val_group.json"

In [ ]:
#import
#warning
import warnings
warnings.filterwarnings('ignore')

#general
import pandas as pd
import numpy as np
from itertools import product
import itertools
import json
from tqdm.notebook import tqdm
from datetime import datetime

import gc

#for predictions
from catboost import CatBoostClassifier, Pool, metrics


In [ ]:
#load data
user_splits=pd.read_parquet(path+'/models/user_splits.parquet')

In [ ]:
# Function to create a new DataFrame for a specific set (train, val, test)
def create_new_df(df, set_type):
    new_data = []

    for index, row in df.iterrows():
        outfit_ids = row[f'{set_type}_outfit_ids']
        unique_outfit_ids, counts = np.unique(outfit_ids, return_counts=True)
        for outfit_id, count in zip(unique_outfit_ids, counts):
            new_data.append({
                'customer_id': row['customer_id'],
                'outfit_id': outfit_id,
                'label': 1
            })

    return pd.DataFrame(new_data)

# Create new DataFrames for train, val, and test
train_df = create_new_df(user_splits, 'train')
val_df = create_new_df(user_splits, 'val')
test_df = create_new_df(user_splits, 'test')


In [ ]:
# Function to create a new DataFrame for a specific set (train, val, test)
def create_new_df_group(df, set_type):
    new_data = []

    for index, row in df.iterrows():
        outfit_ids = row[f'{set_type}_group']
        unique_outfit_ids, counts = np.unique(outfit_ids, return_counts=True)
        for outfit_id, count in zip(unique_outfit_ids, counts):
            new_data.append({
                'customer_id': row['customer_id'],
                'group': outfit_id,
                'label': 1
            })

    return pd.DataFrame(new_data)

# Create new DataFrames for train, val, and test
train_df_group = create_new_df_group(user_splits, 'train')
val_df_group = create_new_df_group(user_splits, 'val')
test_df_group = create_new_df_group(user_splits, 'test')

In [ ]:
orders=pd.read_parquet(path+'/archive/data/orders.parquet')
groups=pd.read_parquet(path+'/archive/data/outfits.parquet')[['id','group']]
orders=orders.merge(groups, left_on='outfit.id', right_on='id', how='left').drop('id', axis=1)
del groups

In [ ]:
def generate_balanced_dataset(df,all_combinations, customer_col='customer_id', outfit_col='outfit_id'):

    # Identify positive examples
    positive_examples = df[[customer_col, outfit_col]]

    # Get negative examples by removing positive examples from all combinations
    negative_examples = pd.merge(all_combinations, positive_examples, on=[customer_col, outfit_col], how='left', indicator=True)
    negative_examples = negative_examples[negative_examples['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Sample negative examples to match the size of positive examples
    negative_examples = negative_examples.sample(n=len(positive_examples), random_state=42)

    # Add a label column to distinguish between positive and negative examples
    positive_examples['label'] = 1
    negative_examples['label'] = 0

    # Combine positive and negative examples
    final_df = pd.concat([positive_examples, negative_examples], ignore_index=True)

    return final_df

In [ ]:
# Get unique customer_ids and outfit_ids
customer_ids = orders['customer.id'].unique()
outfit_ids = orders['outfit.id'].unique()
# Generate all possible combinations of customer_id and outfit_id
all_combinations_outfits = pd.DataFrame(list(itertools.product(customer_ids, outfit_ids)), columns=['customer_id', 'outfit_id'])

In [ ]:
# Generate balanced dataset
train_df = generate_balanced_dataset(train_df,all_combinations_outfits)
val_df = generate_balanced_dataset(val_df,all_combinations_outfits)
test_df = generate_balanced_dataset(test_df,all_combinations_outfits)
del all_combinations_outfits

In [ ]:
# Get unique customer_ids and group
customer_ids = orders['customer.id'].unique()
groups = orders['group'].unique()
# Generate all possible combinations of customer_id and outfit_id
all_combinations_group = pd.DataFrame(list(itertools.product(customer_ids, groups)), columns=['customer_id', 'group'])

In [ ]:
# Generate balanced dataset

train_df_group = generate_balanced_dataset(train_df_group,all_combinations_group,outfit_col='group')
val_df_group = generate_balanced_dataset(val_df_group,all_combinations_group,outfit_col='group')
test_df_group = generate_balanced_dataset(test_df_group,all_combinations_group,outfit_col='group')
del all_combinations_group

In [ ]:
train_df.sample(5)

,customer_id,outfit_id,label
80149,5273,outfit.b4d87ed41b3745d2bea654a2c1d604d4,0
29121,4640,outfit.a4699b3b33b4d99c,1
46260,7395,outfit.fbda7f830faf4caf9b0a085054f22e54,1
32707,5161,outfit.6fc9eb333f5d4d0eab13a0acdeb158ec,1
24748,3875,outfit.ac3e4eefb550471989769e1e4d1683d1,1


In [ ]:
# Concatenate the two DataFrames
combined_df = pd.concat([train_df, val_df])

# Group by customer_id and outfit_id and sum the counts
join_df = combined_df.groupby(['customer_id', 'outfit_id'], as_index=False)['label'].max()

del combined_df

# Concatenate the two DataFrames
combined_df_group = pd.concat([train_df_group, val_df_group])

# Group by customer_id and outfit_id and sum the counts
join_df_group = combined_df_group.groupby(['customer_id', 'group'], as_index=False)['label'].max()

del combined_df_group

In [ ]:
join_df.sample(5)

,customer_id,outfit_id,label
85196,5910,outfit.ae6df823b11943e8b0c9eca3d2723c28,0
46170,3201,outfit.2877791e640e4d87a983b29d5b57a3aa,0
64251,4473,outfit.d968d57e3a3e4c42b75d5acb712cdd42,0
74211,5161,outfit.2470795e1b8a40a581c8259e01e8c37b,0
55206,3845,outfit.02cbb0ff0c1348ce9fb9906805fb9049,0


In [ ]:
outfit_tags=pd.read_parquet(path+'/archive/data/outfit_tags.parquet')
group_tags=pd.read_parquet(path+'/archive/data/outfits.parquet')[['id','group']]
group_tags=outfit_tags.merge(group_tags, on='id').drop('id', axis=1).drop_duplicates()

In [ ]:
# Function to create a dictionary for each id
def create_row(group,id='id'):
    row_dict = {id: group[id].iloc[0]}
    for category_type, categories in group.groupby('category'):
        for i, category in enumerate(categories['tag'], 1):
            row_dict[f'{category_type}_{i}'] = category
    return row_dict

# Apply the function to each group and create a new DataFrame
tags = pd.DataFrame(outfit_tags.groupby('id', group_keys=False).apply(create_row).tolist())
tags.fillna('no muching concept',inplace=True)

tags_group = pd.DataFrame(group_tags.groupby('group', group_keys=False).apply(create_row,'group').tolist())
tags_group.fillna('no muching concept',inplace=True)


In [ ]:
train_df=train_df.merge(tags, left_on='outfit_id', right_on='id').drop('id', axis=1)
val_df=val_df.merge(tags, left_on='outfit_id', right_on='id').drop('id', axis=1)
test_df=test_df.merge(tags, left_on='outfit_id', right_on='id').drop('id', axis=1)
join_df=join_df.merge(tags, left_on='outfit_id', right_on='id').drop('id', axis=1)

In [ ]:
train_df_group=train_df_group.merge(tags_group, on='group')
val_df_group=val_df_group.merge(tags_group, on='group')
test_df_group=test_df_group.merge(tags_group, on='group')
join_df_group=join_df_group.merge(tags_group, on='group')

In [ ]:
del tags_group, groups, tags, outfit_tags

In [ ]:
x_train=train_df.drop(['label'],axis=1)
y_train=train_df['label']
x_val=val_df.drop(['label'],axis=1)
y_val=val_df['label']
x_test=test_df.drop(['label'],axis=1)
y_test=test_df['label']
x_join=join_df.drop(['label'],axis=1)
y_join=join_df['label']

x_train_group=train_df_group.drop(['label'],axis=1)
y_train_group=train_df_group['label']
x_val_group=val_df_group.drop(['label'],axis=1)
y_val_group=val_df_group['label']
x_test_group=test_df_group.drop(['label'],axis=1)
y_test_group=test_df_group['label']
x_join_group=join_df_group.drop(['label'],axis=1)
y_join_group=join_df_group['label']

In [ ]:
categorical_features_indices_outfit = x_train.columns.values.tolist()
categorical_features_indices_group = x_train_group.columns.values.tolist()

In [ ]:
gc.collect()

0

##CatBoost

In [ ]:
def train_catb(x,y,categorical_features_indices,iterations,depth,learning_rate):
  model = CatBoostClassifier(
      iterations=iterations,
      depth=depth,
      learning_rate=learning_rate,
      loss_function='Logloss',
      eval_metric='AUC',
      verbose= 50)
  model.fit(
    x, y,
    cat_features=categorical_features_indices
  )
  return model
def test_catb(algo,x_test,train,group=False):
  x_test['label_predicted']=algo.predict(x_test)
  results=x_test[x_test['label_predicted']==1]
  df_merged = user_splits_df.merge(results, on='customer_id', how='left').groupby('customer_id').agg({
      **{col: 'first' for col in user_splits_df.columns if col != 'customer_id'},
      'label_predicted': lambda x: list(x)
  }).reset_index()
  # Rename the aggregated column to id_prediction
  if group:
    df_merged.rename(columns={'label_predicted': 'group_prediction'}, inplace=True)
  else:
    df_merged.rename(columns={'label_predicted': 'id_prediction'}, inplace=True)
  return x_test,algo,df_merged

def train_and_test_gb(x_train,y_train,categorical_features_indices,x_test,user_splits_df,iterations,depth,learning_rate,group=False):
  algo=train_catb(x_train,y_train,categorical_features_indices,iterations,depth,learning_rate)
  test,algo,df_merged=test_catb(algo,x_test,user_splits_df,group)
  return test,algo,df_merged

In [ ]:
iterations_list=[100,150]
depth_list=[2,5,10]
learning_rate_list=[1,0.5,0.25]

In [ ]:
##Hyper-Parameters tuning for outfit

test_permutations = list(product(iterations_list,depth_list,learning_rate_list))

for iterations,depth,learning_rate in tqdm(test_permutations):
    user_splits_df=user_splits.copy()
    predictions,algo,user_splits_df = train_and_test_gb(x_train,y_train,categorical_features_indices_outfit,x_val,user_splits_df,iterations,depth,learning_rate)
    param_dict = {'iterations':iterations,'depth':depth,'learning_rate':learning_rate}
    user_splits_df, all_dict = evaluate_val_metrics_at_n_outfit(user_splits_df,'CatBoost_tag', n=10,model_params=param_dict)

  0%|          | 0/18 [00:00<?, ?it/s]

0:	total: 141ms	remaining: 13.9s
50:	total: 5.23s	remaining: 5.02s
99:	total: 11.1s	remaining: 0us
0:	total: 139ms	remaining: 13.8s
50:	total: 4.65s	remaining: 4.47s
99:	total: 11.1s	remaining: 0us
0:	total: 136ms	remaining: 13.5s
50:	total: 4.52s	remaining: 4.34s
99:	total: 10.8s	remaining: 0us
0:	total: 225ms	remaining: 22.3s
50:	total: 9.83s	remaining: 9.45s
99:	total: 19.6s	remaining: 0us
0:	total: 242ms	remaining: 23.9s
50:	total: 10.9s	remaining: 10.5s
99:	total: 21.4s	remaining: 0us
0:	total: 227ms	remaining: 22.5s
50:	total: 10.5s	remaining: 10.1s
99:	total: 19s	remaining: 0us
0:	total: 716ms	remaining: 1m 10s
50:	total: 21.9s	remaining: 21s
99:	total: 41.5s	remaining: 0us
0:	total: 749ms	remaining: 1m 14s
50:	total: 20.8s	remaining: 19.9s
99:	total: 41.4s	remaining: 0us
0:	total: 430ms	remaining: 42.6s
50:	total: 19.3s	remaining: 18.6s
99:	total: 39.8s	remaining: 0us
0:	total: 134ms	remaining: 19.9s
50:	total: 4.64s	remaining: 9.01s
100:	total: 9.16s	remaining: 4.45s
149:	tota

In [ ]:
##Hyper-Parameters tuning for group

test_permutations = list(product(iterations_list,depth_list,learning_rate_list))

for iterations,depth,learning_rate in tqdm(test_permutations):
    user_splits_df=user_splits.copy()
    predictions,algo,user_splits_df = train_and_test_gb(x_train_group,y_train_group,categorical_features_indices_group,x_val_group
                                                        ,user_splits_df,iterations,depth,learning_rate,group=True)
    param_dict = {'iterations':iterations,'depth':depth,'learning_rate':learning_rate}
    user_splits_df, all_dict = evaluate_val_metrics_at_n_group(user_splits_df,'CatBoost_tag', n=10,model_params=param_dict)

  0%|          | 0/18 [00:00<?, ?it/s]

0:	total: 146ms	remaining: 14.4s
50:	total: 4.7s	remaining: 4.52s
99:	total: 11s	remaining: 0us
0:	total: 144ms	remaining: 14.3s
50:	total: 4.66s	remaining: 4.47s
99:	total: 9.6s	remaining: 0us
0:	total: 141ms	remaining: 14s
50:	total: 4.51s	remaining: 4.33s
99:	total: 8.88s	remaining: 0us
0:	total: 433ms	remaining: 42.9s
50:	total: 10.4s	remaining: 10s
99:	total: 20.9s	remaining: 0us
0:	total: 229ms	remaining: 22.7s
50:	total: 11s	remaining: 10.6s
99:	total: 20.4s	remaining: 0us
0:	total: 242ms	remaining: 23.9s
50:	total: 9.02s	remaining: 8.66s
99:	total: 19.7s	remaining: 0us
0:	total: 253ms	remaining: 25.1s
50:	total: 22.1s	remaining: 21.2s
99:	total: 42.1s	remaining: 0us
0:	total: 253ms	remaining: 25.1s
50:	total: 21.3s	remaining: 20.4s
99:	total: 41.3s	remaining: 0us
0:	total: 251ms	remaining: 24.8s
50:	total: 20.9s	remaining: 20.1s
99:	total: 40.9s	remaining: 0us
0:	total: 142ms	remaining: 21.2s
50:	total: 6.57s	remaining: 12.8s
100:	total: 11.2s	remaining: 5.41s
149:	total: 15.9s

In [ ]:
# Retrieve the best model parameters from the JSON file for the specified method
best_model_params_outfit = retrieve_best_model_params_from_file_outfit('CatBoost_tag', n=10)
best_model_params_group = retrieve_best_model_params_from_file_group('CatBoost_tag', n=10)

user_splits_df=user_splits.copy()
iterations_outfit=best_model_params_outfit['iterations']
depth_outfit=best_model_params_outfit['depth']
learning_rate_outfit=best_model_params_outfit['learning_rate']

iterations_group=best_model_params_group['iterations']
depth_group=best_model_params_group['depth']
learning_rate_group=best_model_params_group['learning_rate']

# Train the model with the best parameters
predictions,algo,user_splits_df = train_and_test_gb(x_join,y_join,categorical_features_indices_outfit,x_test,user_splits_df,
                                                    iterations_outfit, depth_outfit, learning_rate_outfit)
predictions,algo,user_splits_df = train_and_test_gb(x_join_group,y_join_group,categorical_features_indices_group,x_test_group,user_splits_df,
                                                    iterations_group, depth_group, learning_rate_group,group=True)
# Evaluate the model
user_splits_df, all_dict = evaluate_df_metrics_at_n(user_splits_df,'CatBoost_tag', n=10)

0:	total: 153ms	remaining: 153ms
1:	total: 253ms	remaining: 0us
0:	total: 164ms	remaining: 164ms
1:	total: 278ms	remaining: 0us


,0
id_hit_rate_at_10,0.0
id_precision_at_10,0.0
id_recall_at_10,0.0
id_f1_score_at_10,0.0
id_ndcg_at_10,0.0
group_hit_rate_at_10,0.0
group_precision_at_10,0.0
group_recall_at_10,0.0
group_f1_score_at_10,0.0
group_ndcg_at_10,0.0
